In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="../data",
    train=True,
    download=True,
    transform=ToTensor(),
)
test_data = datasets.FashionMNIST(
    root="../data",
    train=False,
    download=True,
    transform=ToTensor(),
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to('cuda')

In [3]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [4]:
loss_fn = nn.CrossEntropyLoss() # logit + softmax + NLLLoss

In [5]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [6]:
def train_loop(dataloader, model, loss_fn, optimizer):
    model.train()
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        X, y = X.to('cuda'), y.to('cuda')
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to('cuda'), y.to('cuda')
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    accuracy = correct / size
    print(f"Test Error: \n Accuracy: {(100*accuracy):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [7]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.298373  [    0/60000]
loss: 2.284544  [ 6400/60000]
loss: 2.266161  [12800/60000]
loss: 2.260778  [19200/60000]
loss: 2.241892  [25600/60000]
loss: 2.205542  [32000/60000]
loss: 2.229266  [38400/60000]
loss: 2.185787  [44800/60000]
loss: 2.177528  [51200/60000]
loss: 2.143627  [57600/60000]
Test Error: 
 Accuracy: 35.3%, Avg loss: 2.138192 

Epoch 2
-------------------------------
loss: 2.152226  [    0/60000]
loss: 2.138136  [ 6400/60000]
loss: 2.077282  [12800/60000]
loss: 2.096302  [19200/60000]
loss: 2.044892  [25600/60000]
loss: 1.979693  [32000/60000]
loss: 2.022045  [38400/60000]
loss: 1.931508  [44800/60000]
loss: 1.933475  [51200/60000]
loss: 1.860247  [57600/60000]
Test Error: 
 Accuracy: 58.0%, Avg loss: 1.856633 

Epoch 3
-------------------------------
loss: 1.901387  [    0/60000]
loss: 1.861558  [ 6400/60000]
loss: 1.741102  [12800/60000]
loss: 1.782070  [19200/60000]
loss: 1.674021  [25600/60000]
loss: 1.628705  [32000/600